# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,18.62,68,1,5.13,CL,1705954643
1,1,fortuna,40.5982,-124.1573,17.18,87,75,0.89,US,1705954644
2,2,dili,-8.5586,125.5736,27.54,79,100,2.19,TL,1705954644
3,3,myeik,12.4333,98.6000,21.06,68,29,2.09,MM,1705954645
4,4,cidreira,-30.1811,-50.2056,22.68,66,98,6.76,BR,1705954645


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [36]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles = True,
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale= 0.8,
    color = "City",
    alpha =0.6
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df.loc[((city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] < 27)) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"]==0), :]

# Drop any rows with null values
cleaned_df = narrowed_df.dropna()

# Display sample data
cleaned_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,56,san antonio de pale,-1.4014,5.6325,26.99,82,0,3.07,GQ,1705954657
119,119,kabo,7.6994,18.6290,24.58,11,0,2.39,CF,1705954674
146,146,salalah,17.0151,54.0924,23.05,16,0,3.60,OM,1705954681
151,151,chonchi,-42.6128,-73.8092,25.56,36,0,1.50,CL,1705954682
290,290,port augusta,-32.5000,137.7667,25.75,28,0,2.57,AU,1705954719
331,331,kyabe,9.4515,18.9449,26.65,10,0,1.82,TD,1705954730


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cleaned_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
56,san antonio de pale,GQ,-1.4014,5.6325,82,
119,kabo,CF,7.6994,18.6290,11,
146,salalah,OM,17.0151,54.0924,16,
151,chonchi,CL,-42.6128,-73.8092,36,
290,port augusta,AU,-32.5000,137.7667,28,
331,kyabe,TD,9.4515,18.9449,10,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [32]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san antonio de pale - nearest hotel: No hotel found
kabo - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
chonchi - nearest hotel: Hotel Huildín
port augusta - nearest hotel: Hotel Commonwealth
kyabe - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
56,san antonio de pale,GQ,-1.4014,5.6325,82,No hotel found
119,kabo,CF,7.6994,18.6290,11,No hotel found
146,salalah,OM,17.0151,54.0924,16,Muscat International Hotel
151,chonchi,CL,-42.6128,-73.8092,36,Hotel Huildín
290,port augusta,AU,-32.5000,137.7667,28,Hotel Commonwealth
331,kyabe,TD,9.4515,18.9449,10,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles=True,
    frame_width = 1000,
    frame_height = 500,
    size = "Humidity",
    scale= 1.5,
    color = "City",
    alpha =0.5,
    hover_cols=["Country", "Hotel Name"]  
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)